# Routing

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
os.environ['LANGSMITH_PROJECT']="ai-agents-rags"
os.environ['LANGSMITH_TRACING']="true"

## Logical & Semantic Routing
https://python.langchain.com/docs/how_to/routing/#routing-by-semantic-similarity
![Semantic Routing](./logical-semantic-routing.png)

In [4]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )

# LLM with function call 
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm = llm.with_structured_output(RouteQuery)

# Prompt 
system = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
router = prompt | structured_llm

### Structured Output
![Function Calling](./function-calling.png)

In [6]:
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = router.invoke({"question": question})

In [8]:
result, result.datasource

(RouteQuery(datasource='python_docs'), 'python_docs')

In [9]:
def choose_route(result):
    if "python_docs" in result.datasource.lower():
        ### Logic here 
        return "chain for python_docs"
    elif "js_docs" in result.datasource.lower():
        ### Logic here 
        return "chain for js_docs"
    else:
        ### Logic here 
        return "golang_docs"

from langchain_core.runnables import RunnableLambda

full_chain = router | RunnableLambda(choose_route)

In [10]:
full_chain.invoke({"question": question})


'chain for python_docs'

### Semantic Routing
![semantic-routing](./semantic-routing.png)

In [12]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Two prompts
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

# Embed prompts
embeddings = OpenAIEmbeddings()
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# Route question to prompt 
def prompt_router(input):
    # Embed question
    query_embedding = embeddings.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # Chosen prompt 
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI()
    | StrOutputParser()
)

print(chain.invoke("What's a black hole"))

Using PHYSICS
A black hole is a region in space where gravity is so strong that nothing, not even light, can escape from it. This phenomenon occurs when a massive star collapses in on itself. The gravitational pull is so intense that it creates a point of infinite density called a singularity at its center. The boundary surrounding the singularity is called the event horizon, beyond which nothing can escape. Black holes are fascinating objects in the universe that continue to be studied by physicists and astronomers.


In [13]:
print(chain.invoke("Explain ODE"))

Using PHYSICS
ODE stands for Ordinary Differential Equation. It is an equation that involves a function of one variable and its derivatives. These equations are commonly used in physics to model the behavior of physical systems over time. ODEs can describe how a system changes in response to various factors and can be solved to predict the future behavior of the system. They are an important tool in understanding complex dynamical systems in physics.


In [14]:
print(chain.invoke("ORdinary differential equations belongs to the subject of Mathematics or Physics?"))

Using MATH
Ordinary differential equations belong to the subject of mathematics. They are a fundamental topic in mathematics, specifically in the branch of calculus and differential equations. While they have many applications in physics and other sciences, they are primarily studied and developed within the mathematical framework.
